# Description of the Economic Model 
The economic model uses the latest econonic data from Belgium to estimate the impact of the pandemic.
It is to be run in parallel to the SEIRS Network model and could easily be merged into it. 

In [8]:
import numpy as np
import economic_model

To launch it, simply create an `EconomicModel` object.

In [9]:
EcoMod = economic_model.EconomicModel

## Available data sources
At the moment, three major sources of data are available and loaded:
* The Sectoral Data (Sectoral_data.xlsx) contains the value added and the employment by each economic sector of Belgium.
* The "Staff distribution by sector.xlsx" is recent data (recieved on April 25th) covering 70% of Belgian workforce that provide the breakdown of people working from home, from their usual workplace, a mix of both or are temporarily unemployed or absent. This data is taken as the maximal impact from confinment measures.
* Demographic data from https://statbel.fgov.be/en/themes/population/structure-population used to estimate the population of children and students (people below 20 years old), retired people (70 y.o. and above) and the full population. 
The demographic and sectoral data are also used to estimate the amount of non-working people between 20 y.o. and 70 y.o.

The `LoadInputs` function recovers and stores data from theses sources.
Three variables hold these data:

```python
Inputs = 'Industry breakdown'      # Description of industry classes
         'Value Added'             # Economic value added by Belgian workers (per year)
         'Employment'              # Number of employee per industry
         'Value Added per Employe' # Value added per employee
         'Employment fraction'     # Ratio of employees in each industry with respect to total number of workers.

ReferenceNumbers = 'Population'         # Full population of Belgium
                   'Student population' # Population of "students", i.e. Belgians below 20.
                   'Retired population' # Population of "retired", i.e. Belgians above 70.
                   'Working population' # Population of "working" Belglian without pandemic.

Adaptation = 'Work at home'             # Employees working from home
             'Mix home - office'        # Employees using a mix of work from home and at the workplace
             'Work at work'             # Employees working from their workplace
             'Temporary Unemployed'     # Employees temporarily unemployed
             'Absent'                   # Employees absent from work
             'Date of Survey'           # Date of the reception of the survey`
```
These informations are stored in python dictionnary and can be accessed from the model as below: 

In [10]:
EcoMod.LoadInputs(EcoMod)
print('The stored population of Belgium is ' + str(EcoMod.ReferenceNumbers['Population']) + ' inhabitants')

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\Sectoral_data.xlsx'

## Building an economical model
The economic model assesses a large network of individuals from the SEIRS Network Model. Once each individual is assigned a health status, they can receive an economical status which is the combination of an `Occupation` and a `State`.
This status could be integrated to SEIRS Network but, at the moment, it is simply generated in a stand-alone 2D matrix. 

## Addind an occupation to a network of individuals
The network of individuals can be assigned to an occupation and a state.
The occupation will be any of the following, based on demographic data and 2018 employment census:
* [-2] Student (0-20 yo)
* [-1] Retired (70+ yo)
* [0] Non-working (active population that do not work or seek employment)
* [1 - 37] Working or work seeker in one of 37 economical sectors

The state of non-working population (students, retired or non-working) can be:
* [-1] Confined
* [0] Unconfined
And the state of working population can be:
* [1] Working from home
* [2] Mix of working from home and at work (office/factory/etc.)
* [3] Working at work (office/factory/etc.)
* [4] Temporarily unemployed
* [5] Absent

The function `AssignOccupation` returns a Status Matrix that assigns both an occupation and a state of work randomly, according to current data. Each line correspond to one individual. Column zero holds the `Occupation` and column 1 holds the `State`.

In [4]:
PopulationMatrix = EcoMod.AssignOccupation(EcoMod,N = 2000, ConfinementPolicy = True)
print(PopulationMatrix[1:10,:])

AttributeError: type object 'EconomicModel' has no attribute 'ReferenceNumbers'

Occupation information about the population can be extracted with the function `ComputeOccupation`. The number of confined, unconfined, working and unemployed individuals are returned. 

In [ ]:
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))

## Added Value
The `Added Value` gives an indication of the state of the economy. Industries need different amount of work to generate value, which is reflected in the added value. 
The function `ComputeValueAdded` computes the value added by the workers of a network by multiplying the `Value added by a Belgian worker` by the number of active workers in that specific industry. At the moment, no distinction is made wheter the work is performed at home or at the usual workplace.

In [ ]:
print("% 10.2f euros"% (EcoMod.ComputeValueAdded(EcoMod,PopulationMatrix))) 

## Altering the confinement policy
In order to assess different scenarios, it is possible to alter restrictions on selected sectors of the population with the function `ChangeConfinementPolicy`. 
A call to `ChangeConfinementPolicy allows to set non-working individuals (students, retired, non-workers) to Confined or Unconfined status or to set a specific economy sector to a new confinement policy, between [0 - 1]. A policy of 0 maximizes the work from home and unemployment while a policy of 1 sends everyone to work (office/factory/etc.). Any number in between offers a linear tradeof.

For example, lifting the confinement on children and students can be done as follow:

In [ ]:
PopulationMatrix = EcoMod.ChangeConfinementPolicy(EcoMod,PopulationMatrix,-2, 1)
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))

Then opening back all retails (economic sector number 19):

In [ ]:
PopulationMatrix = EcoMod.ChangeConfinementPolicy(EcoMod,PopulationMatrix,19, 1)
print(EcoMod.ComputeOccupation(EcoMod,PopulationMatrix))
print("% 10.2f euros"% (EcoMod.ComputeValueAdded(EcoMod,PopulationMatrix))) 

Therefore, by reopening the shops, the daily added value of the network increased by about 10% (numbers will differ due to the random initialization of the network).

## State of development of the Economic Model

The model is still very basic, with only two objective functions (`ComputeOccupation` and `ComputeValueadded`).

Input-Output tables are available in the DATA folder, but not loaded in the model. 

No modification to the SEIRS model has been made to couple it with the economic model. However, as of today (April 27th), the link can be made by adding two attributes to each individual in the SEIRS: Occupation and Status. 
